first we import all whats needed

In [16]:
import cv2                                  # for video processing and drawing
import numpy as np                          # for all numerical operations
from collections import OrderedDict         # Keeps track of objects in the order they were added
from scipy.spatial import distance as dist  # Computes pairwise distances between points (used for tracking)

now all needed pathes were added

In [17]:
video_path = "../videos/CarsOnAHighway_30fps.mp4"           # the video we do use
prototxt_path = "../models/deploy.prototxt"                 # the network architecture of MobileNetSSD 
model_path = "../models/mobilenet_iter_73000.caffemodel"    # the trained weights of MobileNetSSD 
output_path = "output_tracked.mp4"                          # the video we do want to create

Load MobileNetSSD - what happens here is that the architecture information and the calculated weights are taken and a model is created out of that.
A list is created with all detectable opbjects to have an ID <-> name relation later

In [19]:
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

now the improved centraid tracker is created

In [20]:
# Improved Centroid Tracker
class CentroidTracker:
    def __init__(self, maxDisappeared=50):      # C O N S T R U C T O R:
        self.nextObjectID = 0                   # here the next upcoming object ID is storeded (used in register())
        self.objects = OrderedDict()            # a dictionary is created for all existing objects - ID:[X-pos, Y-Pos]
        self.disappeared = OrderedDict()        # a dictionary is created - info how long a object was invisible ID:frames
        self.maxDisappeared = maxDisappeared
        
    # register a newly found car 
    # centroid - the x / y possition of the objet as an Humpy Array
    def register(self, centroid):                   
        self.objects[self.nextObjectID] = centroid  # create a dictonary entry for the object and store its position
        self.disappeared[self.nextObjectID] = 0     # create a dictonary entry for the objects dissapearance timer and set it to Zero
        self.nextObjectID += 1                      # calculate the next object ID

    # if the car is gone, kick it out
    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    # update each object separatly by stepping through all objects detected in n-dimensional array rects
    def update(self, rects):
        if len(rects) == 0:                                           # if no object was found you can delete them all
            for objectID in list(self.disappeared.keys()):            # for all remaining objects
                self.disappeared[objectID] += 1                       # increment the dissapearance counter
                if self.disappeared[objectID] > self.maxDisappeared:  # if gobe for to long
                    self.deregister(objectID)                         # delete this object
            return self.objects                                       # jump back

        inputCentroids = np.zeros((len(rects), 2), dtype="int")       # create a numpy array element for each found objects and its coordinates (reservaze space only)
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        if len(self.objects) == 0:
            for i in range(len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
                usedRows.add(row)
                usedCols.add(col)

            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        return self.objects

tracker = CentroidTracker()

Open the video and store it

In [21]:
# Open video
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

create a VideoWriter for output

In [22]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

Now do the magic

In [23]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    rects = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.4:
            idx = int(detections[0, 0, i, 1])
            if CLASSES[idx] in ["car", "bus", "person"]:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                rects.append(box.astype("int"))

    objects = tracker.update(rects)

    for (objectID, centroid) in objects.items():
        text = f"ID {objectID}"
        cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

    out.write(frame)

    cap.release()
out.release()

print("Annotated video with persistent IDs saved as output_tracked.mp4")
print("To display in Jupyter, use:")
print("from IPython.display import Video; Video('output_tracked.mp4', embed=True)")

Annotated video with persistent IDs saved as output_tracked.mp4
To display in Jupyter, use:
from IPython.display import Video; Video('output_tracked.mp4', embed=True)
